# *Algo 3.1 for the top down model*

In [ ]:
%pip install numpy scipy matplotlib

In [ ]:
import logging
import numpy as np
from scipy.special import hyp1f1, iv
from scipy.optimize import brentq, minimize_scalar
from scipy.integrate import cumulative_trapezoid
from scipy.optimize import toms748
from scipy.integrate import simpson as simps
from scipy.interpolate import interp1d
from typing import Optional, Tuple, Dict, List
from dataclasses import dataclass
import warnings
import time
import math

ModelParams = Dict[str, float]

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('default_simulation.log')
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class SimulationConfig:
    """Configuration for numerical methods and tolerances"""
    # Numerical methods and tolerances
    # Constants for numerical stability and convergence
    eps_zero: float = 1e-12
    eps_small: float = 1e-8
    # Maximum iterations for numerical methods
    max_iterations: int = 1000
    integration_points: int = 500
    root_finding_maxiter: int = 100
    # Number of grid points for numerical integration and interpolation
    grid_points: int = 500
    # Tolerance for numerical methods
    tolerance_scale: float = 1e-6

config = SimulationConfig()

In [21]:
def validate_parameters(model_params: ModelParams) -> bool:
    """Validate CIR parameters and check Feller condition"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"] 
        sigma = model_params["sigma"]
        
        assert kappa > 0, "kappa must be positive"
        assert theta > 0, "theta must be positive" 
        assert sigma > 0, "sigma must be positive"
        
        feller_condition = 2 * kappa * theta >= sigma**2
        if not feller_condition:
            logger.warning(f"Feller condition violated: 2kappa*theta={2*kappa*theta:.6f} < sigma^2={sigma**2:.6f}")
            
        logger.info(f"Parameters: κ={kappa:.6f}, θ={theta:.6f}, σ={sigma:.6f}")
        logger.info(f"Feller condition {'satisfied' if feller_condition else 'violated'}")
        
        return feller_condition
        
    except Exception as e:
        logger.error(f"Parameter validation failed: {e}")
        raise

In [ ]:
def _hyp1f1(a: float, b: float, z: float, method: str = 'primary') -> float:
    """Hypergeometric function evaluation with fallbacks"""
    try:
        # Primary method
        if method == 'primary' and abs(z) < 100 and abs(a) < 50:
            result = hyp1f1(a, b, z)
            if np.isfinite(result) and abs(result) < 1e10:
                return result
        
        # # Fallback for large arguments using series truncation
        # if abs(z) > 100 or abs(a) > 50:
        #     logger.debug(f"Using series approximation for hyp1f1({a:.3f}, {b:.3f}, {z:.3f})")
        #     # For large z, use asymptotic expansion if a is negative integer-like
        #     if a < 0 and abs(a - round(a)) < config.eps_small:
        #         n = int(abs(round(a)))
        #         if n < 20:  # Finite series
        #             result = sum((a + k) * z**k / math.factorial(k) / (b + k) 
        #                        for k in range(n + 1))
        #             return result
        
        # Fallback resort: clip extreme values
        a_clip = np.clip(a, -50, 50)
        z_clip = np.clip(z, -100, 100)
        result = hyp1f1(a_clip, b, z_clip)
        
        if not np.isfinite(result):
            logger.warning(f"hyp1f1 returned non-finite: a={a}, b={b}, z={z}")
            return 1.0  
            
        return result
        
    except Exception as e:
        logger.warning(f"hyp1f1 evaluation failed: {e}, using fallback")
        return 1.0

In [ ]:
def hyp1f1_derivative_a(a: float, b: float, z: float, delta: float = None) -> float:
    """Numerical derivative of hypergeometric function"""
    if delta is None:
        delta = max(config.eps_small, abs(a) * config.tolerance_scale)
    
    try:
        f_plus = _hyp1f1(a + delta, b, z)
        f_minus = _hyp1f1(a - delta, b, z)
        derivative = (f_plus - f_minus) / (2 * delta)
        
        if not np.isfinite(derivative):
            # Try smaller delta
            delta *= 0.1
            f_plus = _hyp1f1(a + delta, b, z)
            f_minus = _hyp1f1(a - delta, b, z)
            derivative = (f_plus - f_minus) / (2 * delta)
            
        return derivative if np.isfinite(derivative) else 0.0
        
    except Exception as e:
        logger.warning(f"Derivative computation failed: {e}")
        return 0.0

In [ ]:
def laplace_transform_G(H: float, y: float, model_params: ModelParams) -> float:
    """Laplace transform computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        a = H / kappa
        b = 2 * kappa * theta / sigma**2
        z1 = 2 * kappa * y / sigma**2
        z2 = 2 * kappa * H / sigma**2
        
        logger.debug(f"Laplace transform: a={a:.6f}, b={b:.6f}, z1={z1:.6f}, z2={z2:.6f}")
        
        numerator = _hyp1f1(a, b, z1)
        denominator = _hyp1f1(a, b, z2)
        
        if abs(denominator) < config.eps_zero:
            logger.warning("Near-zero denominator in Laplace transform")
            return 0.99  # Safe fallback
            
        result = numerator / denominator
        
        # Stability check
        # if not (0 <= result <= 1):
        #     logger.warning(f"Laplace transform out of bounds: {result:.6f}")
        #     result = np.clip(result, config.eps_zero, 1 - config.eps_zero)
            
        return result
        
    except Exception as e:
        logger.error(f"Laplace transform failed: {e}")
        return 0.5  # Conservative fallback

In [ ]:
def compute_coefficients(y: float, H: float, model_params: ModelParams, n_roots: int = 15) -> Tuple[List[float], List[float]]:
    """Compute eigenfunction expansion coefficients as given by eqn 20 in section 4.2. root finding given in appendix B of the paper"""
    start_time = time.time()
    
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        c_bar = 2 * kappa * theta / sigma**2
        H_bar = 2 * kappa * H / sigma**2
        y_bar = 2 * kappa * y / sigma**2

        if c_bar < 1 or H_bar < c_bar:
            logger.error(f"Invalid parameters: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
            return [], []
        
        logger.debug(f"Computing coefficients: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
        
        def objective(alpha):
            return _hyp1f1(alpha, c_bar, H_bar)
        
        roots = []
        n = 0
        alpha_prev = 0

        # Find the first root in (-c_bar / H_bar, 0)
        left = -c_bar / H_bar
        right = 0
        try:
            alpha_curr = brentq(objective, left, right)
            roots.append(alpha_curr)
        except Exception as e:
            logger.error(f"Failed to find first root: {e}")
            return [], []

        # Find subsequent roots
        while len(roots) < n_roots:
            n += 1
            # Predict the next root using extrapolation
            alpha_hat = 2 * alpha_curr - alpha_prev

            # Inner loop to bracket the root
            bracket_found = False
            for attempt in range(20):  
                try:
                    test_val = objective(alpha_hat - 1)
                    if np.sign(test_val) == (-1)**n:
                        alpha_hat = alpha_hat - 1
                    else:
                        bracket_found = True
                        break
                except:
                    alpha_hat = alpha_hat - 1
                    if attempt > 10:  # Give up after many attempts
                        break
            
            if not bracket_found:
                logger.warning(f"Could not bracket root {n+1}, stopping at {len(roots)} roots")
                break
                
            try:
                alpha_next = brentq(objective, alpha_hat - 1, alpha_hat)
                roots.append(alpha_next)
                alpha_prev = alpha_curr
                alpha_curr = alpha_next
            except Exception as e:
                logger.warning(f"Failed to find root {n+1}: {e}, stopping at {len(roots)} roots")
                break
        
        if len(roots) < 5:
            logger.error(f"Insufficient roots found: {len(roots)}")
            return [], []
        
        # Compute coefficients
        eta_n, beta_n = [], []
        for alpha in roots:
            try:

                if abs(alpha) <= config.eps_zero:
                    logger.warning(f"Skipping near-zero alpha={alpha:.6f}")
                    eta_n.append(0.0)
                    beta_n.append(0.0)
                    continue
                
                eta = -kappa * alpha
                eta_n.append(eta)
                
                numerator = _hyp1f1(alpha, c_bar, y_bar)
                derivative = hyp1f1_derivative_a(alpha, c_bar, H_bar)
                
                if abs(derivative) < config.eps_zero:
                    logger.warning(f"Near-zero derivative for alpha={alpha:.6f}")
                    beta = 0.0
                else:
                    beta = -numerator / (alpha * derivative)

                if abs(alpha*derivative) < config.eps_small:
                    logger.warning(f"Near-zero product of alpha and derivative for alpha={alpha:.6f}")
                    beta = 0.0
                elif not np.isfinite(beta):
                    logger.warning(f"Non-finite beta for alpha={alpha:.6f}, setting to 0.0")
                    beta = 0.0
                else:
                    beta = -numerator / (alpha * derivative)
                    
                beta_n.append(beta)
                
            except Exception as e:
                logger.warning(f"Coefficient computation failed for alpha={alpha:.6f}: {e}")
                eta_n.append(0.0)
                beta_n.append(0.0)
        
        # Filter out problematic coefficients
        valid_indices = [i for i, (eta, beta) in enumerate(zip(eta_n, beta_n)) 
                        if np.isfinite(eta) and np.isfinite(beta)]
        
        eta_n = [eta_n[i] for i in valid_indices]
        beta_n = [beta_n[i] for i in valid_indices]
        
        logger.debug(f"Computed {len(eta_n)} valid coefficients in {time.time() - start_time:.3f}s")
        
        return eta_n, beta_n
        
    except Exception as e:
        logger.error(f"Coefficient computation failed: {e}")
        return [], []

In [ ]:
def compute_survival_probability_p1(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Compute survival probability"""
    try:
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients available, using fallback")
            return 0.5
        
        p1 = sum(beta * np.exp(-eta * tau) for eta, beta in zip(eta_n, beta_n) 
                if np.isfinite(beta) and np.isfinite(eta))
        
        # Stability bounds
        p1 = np.clip(p1, config.eps_zero, 1 - config.eps_zero)
        
        logger.debug(f"Survival probability: {p1:.6f}")
        return p1
        
    except Exception as e:
        logger.error(f"Survival probability computation failed: {e}")
        return 0.5

In [ ]:
def cir_transition_density_g(x: float, y: float, t: float, model_params: ModelParams) -> float:
    """CIR transition density computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        if t <= 0 or x < 0 or y < 0:
            return 0.0
            
        q = (2 * kappa * theta / sigma**2) - 1
        exp_term = np.exp(-kappa * t)
        a = 2 * kappa / (sigma**2 * (1 - exp_term))
        b = a * exp_term
        
        by = b * y + config.eps_zero
        ax = a * x + config.eps_zero
        
        # Compute in log space for stability
        log_factor = np.log(a) - (ax + b * y)
        log_power = (q / 2) * (np.log(ax) - np.log(by))
        
        # Bessel function argument
        bessel_arg = 2 * np.sqrt(a * b * x * y)
        if bessel_arg > 100:  # Use asymptotic approximation
            log_bessel = bessel_arg - 0.5 * np.log(2 * np.pi * bessel_arg)
        else:
            bessel_val = iv(q, bessel_arg)
            if bessel_val <= 0:
                return 0.0
            log_bessel = np.log(bessel_val)
        
        log_density = log_factor + log_power + log_bessel
        
        if log_density > 50:  # Prevent overflow
            return 0.0
            
        return np.exp(log_density)
        
    except Exception as e:
        logger.debug(f"CIR density computation failed: {e}")
        return 0.0

In [ ]:
def select_dominating_intensity_H(y: float, model_params: ModelParams, 
                              t_C: float = 1.0, t_eta: float = 1.8, buffer: float = 0.01) -> float:
    """Find optimal threshold H"""
    try:
        theta = model_params["theta"]
        
        H_max = max(y * 3, theta * 2, 1.0)
        H_min = max(y + config.eps_small, theta + buffer, config.eps_small)
        
        def objective(H):
            try:
                G = laplace_transform_G(H, y, model_params)
                
                numerator = H * (t_C + G * t_eta)
                denominator = 1 - G
                
                if denominator < config.eps_zero:
                    return 1e10
                    
                return numerator / denominator
                
            except:
                return 1e10
        
        result = minimize_scalar(objective, method='brent')
        
        if not result.success:
            logger.warning("H optimization failed, using heuristic")
            H_star = max(y * 1.5, theta * 1.2)
        else:
            H_star = result.x
        
        # Apply safety constraints
        H_final = max(H_star, y + config.eps_small, theta + buffer)
        
        logger.debug(f"Optimal H: {H_final:.6f} (y={y:.6f})")
        return H_final
        
    except Exception as e:
        logger.error(f"H optimization failed: {e}")
        return max(y * 1.5, theta * 1.2, 1.0)


In [ ]:
def sample_conditional_intensity_from_f(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample from conditional transition density"""
    try:
        theta = model_params["theta"]
        nu_scale = max(y, theta, 0.1)
        x_max = max(nu_scale * 5, 10.0)  # Reasonable upper bound
        
        n_points = config.integration_points
        x_grid = np.linspace(config.eps_zero, x_max, n_points, endpoint=False)
        
        # Compute transition density
        g_vals = np.array([cir_transition_density_g(x, y, tau, model_params) for x in x_grid])
        
        # Compute convolution integral (simplified for robustness)
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        eta_n = np.array(eta_n)
        beta_n = np.array(beta_n)

        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        def u(s):
            s = np.atleast_1d(s)
            return np.sum(beta_n[:, None] * eta_n[:, None] * np.exp(-eta_n[:, None] * s[None, :]), axis=0)

        
        conv_vals = np.empty_like(x_grid)

        for i, xi in enumerate(x_grid):
            g_shift = np.array([
                cir_transition_density_g(xi, H, tau - sj, model_params)
                for sj in s_grid
            ])
            conv_vals[i] = cumulative_trapezoid(g_shift * u(s_grid), s_grid)[-1]

        p1 = compute_survival_probability_p1(y, H, tau, model_params)
            
        f_vals = (g_vals - conv_vals) / p1
        f_vals = np.clip(f_vals, config.eps_zero, None)  # Avoid negative densities
        f_vals /= np.sum(f_vals)  # Normalize

        if np.sum(f_vals) <= config.eps_zero:
            logger.warning("Sampling failed, using fallback")
            return max(y * np.random.uniform(0.5, 1.5), config.eps_small)
        

        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        # cdf_vals = np.cumsum((f_vals[:-1] + f_vals[1:]) * 0.5 * np.diff(x_grid))
        cdf_vals = np.concatenate(([0.0], cdf_vals))
        total_mass = cdf_vals[-1]
        if total_mass <= 0:
            return y
        cdf_vals /= total_mass

        if cdf_vals[-1] > config.eps_zero:
            cdf_vals /= cdf_vals[-1]
        else:
            # Uniform fallback
            cdf_vals = np.linspace(0, 1, len(cdf_vals))
        
        # Ensure monotonicity
        for i in range(1, len(cdf_vals)):
            cdf_vals[i] = max(cdf_vals[i], cdf_vals[i-1])
        
        # Sample
        u = np.random.uniform()
        try:
            inv_cdf = interp1d(cdf_vals, x_grid, bounds_error=False, 
                             fill_value=(x_grid[0], x_grid[-1]))
            sample = float(inv_cdf(u))
        except:
            # Linear interpolation fallback
            idx = np.searchsorted(cdf_vals, u)
            idx = min(idx, len(x_grid) - 1)
            sample = x_grid[idx]
        
        # Final bounds check
        sample = max(sample, config.eps_small)
        
        logger.debug(f"Sampled intensity: {sample:.6f}")
        return sample
        
    except Exception as e:
        logger.error(f"Conditional sampling failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)


In [ ]:
def sample_hitting_time_from_v(nu_t: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample hitting time"""
    try:
        eta_n, beta_n = compute_coefficients(nu_t, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients for hitting time, using uniform")
            return np.random.uniform(0, tau)
        
        P = 1 - compute_survival_probability_p1(nu_t, H, tau, model_params)
        P = max(P, config.eps_zero)
        
        U = np.random.uniform(0, P)
        
        def objective(s):
            integral = sum(beta * (1 - np.exp(-eta * s)) for beta, eta in zip(beta_n, eta_n)
                          if eta > config.eps_zero)
            return integral - U

        try:
            result = brentq(objective, 0, tau)
            return min(result, tau)
        except:
            logger.warning("Hitting time root finding failed, using approximation")
            return np.random.uniform(0, tau * 0.8)
            
    except Exception as e:
        logger.error(f"Hitting time sampling failed: {e}")
        return np.random.uniform(0, tau * 0.5)

In [ ]:
def sample_loss() -> float:
    """Sample loss from a uniform distribution as per the original model"""
    u = np.random.uniform()
    if u < 0.5:
        return 0.96
    else:
        return 0.24

In [ ]:
def simulate_next_default_step(
    t_prev: float,
    lambda_prev: float,
    T_max: float,
    model_params: ModelParams
) -> Optional[Tuple[float, float]]:
    """
    Simulates a single default step in the top-down model.

    Parameters:
        t_prev: float
            Previous default time T_{n-1}
        lambda_prev: float
            Intensity after last jump, λ_{T_{n-1}} 
        T_max: float
            Final time horizon for simulation
        model_params: dict
            Contains CIR and model parameters: kappa, theta, sigma, gamma

    Returns:
        Optional[Tuple[float, float]]:
            If a default occurs before T_max:
                (T_n, ν_{T_n})
            Else:
                None (no more defaults)
    """
    try:
        # STEP 1: Initialize y := λ_{T_{n-1}}, t := T_{n-1}
        y = max(lambda_prev, config.eps_small) # Ensure y is positive
        t = t_prev 
        
        max_inner_iterations = 50  
        
        for iteration in range(max_inner_iterations):
            # STEP 2: Select H>y
            H = select_dominating_intensity_H(y, model_params)
            logger.info(f"Iteration {iteration + 1}: Selected H={H:.6f} for y={y:.6f}")
            
            # STEP 3: Sample τ ∼ Exp(H), propose next interarrival time
            tau = np.random.exponential(1/H)
            logger.debug(f"Sampled interarrival time τ={tau:.6f} from Exp(H) with H={H:.6f}")
            
            # STEP 4: Compute survival probability p1 = P(ν stays below H up to τ)
            p1 = compute_survival_probability_p1(y, H, tau, model_params)
            logger.info(f"Computed survival probability p1={p1:.6f} for y={y:.6f}, H={H:.6f}, τ={tau:.6f}")
            
            # STEP 5: First rejection test with u1 ∼ Unif(0,1)
            u1 = np.random.uniform()
            
            if u1 > p1:
                # Process hits H → sample σ_H from hitting time density
                logger.info(f"Process hits boundary H={H:.6f} at t={t:.6f}, sampling hitting time")
                sigma_H = sample_hitting_time_from_v(y, H, tau, model_params)
                t += sigma_H
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                    
                y = H  # Update intensity and repeat
                continue
            else:
                # Process survives to τ
                t += tau
                logger.info(f"Process survives to t={t:.6f}, sampling conditional intensity")
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                
                # STEP 6: Sample ν_{T_n} from conditional density f(.; ν_t, H, τ)
                nu_Tn = sample_conditional_intensity_from_f(y, H, tau, model_params)
                logger.debug(f"Sampled conditional intensity ν_{t}={nu_Tn:.6f} at t={t:.6f}")
                
                # STEP 7: Second rejection test using u2 ∼ Unif(0,1)
                u2 = np.random.uniform()
                
                if u2 <= nu_Tn / H:
                    # Accept default at T_n = t
                    # ell_n = sample_loss()  # Draw loss ℓ_n ∼ loss_distribution()
                    
                    # logger.debug(f"Default at t={t:.6f}, ν={nu_Tn:.6f}, loss={ell_n:.6f}")
                    logger.info(f"Default accepted at t={t:.6f}, ν={nu_Tn:.6f}.")
                    return (t, nu_Tn)
                else:
                    # Reject - update ν_t and go back to step 2
                    y = nu_Tn
                    continue
        
        logger.warning(f"Maximum inner iterations reached at t={t:.6f}")
        return None
        
    except Exception as e:
        logger.error(f"Error in simulate_next_default_step: {e}")
        return None

def simulate_defaults(
    lambda_0: float,
    T_max: float,
    model_params: ModelParams,
    max_defaults: int = 1000
) -> List[Tuple[float, float]]:
    """
    Main simulation function that generates a sequence of default times and losses.
    
    Parameters:
        lambda_0: float
            Initial intensity λ_0
        T_max: float
            Time horizon
        model_params: ModelParams
            Dictionary containing kappa, theta, sigma, gamma
        max_defaults: int
            Maximum number of defaults to simulate
            
    Returns:
        List[Tuple[float, float]]: List of (default_time, loss) pairs
    """
    logger.info(f"Starting default simulation: λ_0={lambda_0:.6f}, T_max={T_max:.2f}")
    
    # Validate parameters
    if not validate_parameters(model_params):
        logger.warning("Proceeding despite Feller condition violation")
    
    defaults = []
    t = 0.0
    lam = max(lambda_0, config.eps_small)
    # gamma = model_params.get("gamma", 2.99)  # Default jump size
    gamma = model_params["gamma"]
    start_time = time.time()
    
    try:
        while t < T_max and len(defaults) < max_defaults:
            result = simulate_next_default_step(t, lam, T_max, model_params)
            
            if result is None:
                break
                
            Tn, nu_Tn = result
            ell_n = sample_loss()
            lam = nu_Tn + gamma * ell_n  # Update intensity at default with jump
            t = Tn
            defaults.append((t, lam, ell_n))
            
            if len(defaults) % 10 == 0:
                logger.info(f"Generated {len(defaults)} defaults, t={t:.4f}, λ={lam:.6f}")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Simulation completed: {len(defaults)} defaults in {elapsed_time:.2f}s")
        
        return defaults
        
    except KeyboardInterrupt:
        logger.info("Simulation interrupted by user")
        return defaults
    except Exception as e:
        logger.error(f"Critical simulation error: {e}")
        return defaults

In [ ]:
# paper parameters for testing
model_params = {
    "kappa": 2.62,    # Mean reversion speed
    "theta": 1.61,    # Long-term mean
    "sigma": 0.62,    # Volatility
    "gamma": 0.99     # Jump size multiplier
}

defaults = simulate_defaults(
    lambda_0=0.7,
    T_max=5,
    model_params=model_params,
    max_defaults=100
)

print(f"Generated {len(defaults)} defaults:")
for i, (t, lam, loss) in enumerate(defaults): 
    print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")


In [ ]:
L = sum(loss for _, _, loss in defaults)
print(f"Total loss: {L:.6f}")

In [ ]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 0.99     # Jump size multiplier - paper's value is 2.99 but we use 0.99 for stability 
    # !TODO: Check why Laplace transform blows up with gamma >~ 1.5. 
    # All further calculations of survival probabilities and sampling from f,v are unstable.
}

L_5 = []

for _ in range(10):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=100
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 10 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")



In [ ]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 2.99     # Jump size multiplier 
}

L_5 = []

for _ in range(5):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=10
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 5 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")



In [ ]:
y =[0.7, 1.0, 1.5, 2.0, 2.5]
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 0.99     # Jump size multiplier - paper's value is 2.99 but we use 0.99 for stability 
    # !TODO: Check why Laplace transform blows up with gamma >~ 1.5. 
    # All further calculations of survival probabilities and sampling from f,v are unstable.
}
H = [select_dominating_intensity_H(yi, model_params) for yi in y]

import pandas as pd
df = pd.DataFrame({
    'y': y,
    'H': H
})
print("Selected H values for different y:")
print(df)

In [ ]:
laplaces= [laplace_transform_G(Hi, yi, model_params) for Hi, yi in zip(H, y)]

In [ ]:
print("Laplace transforms for different (H, y) pairs:")
for yi, Hi, laplace in zip(y, H, laplaces):
    print(f"  y={yi:.2f}, H={Hi:.2f} => G(H, y)={laplace:.6f}")
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(y, H, marker='o', label='H(y)', color='blue')
plt.title('Dominating Intensity H as a Function of y')
plt.xlabel('y (Intensity)')
plt.ylabel('H (Dominating Intensity)')
plt.grid()
plt.legend()
# plt.savefig('H_vs_y.png')
plt.show()

In [ ]:
plt.plot(y, laplaces, marker='o', label='Laplace Transform G(H, y)', color='orange')
plt.title('Laplace Transform G(H, y) as a Function of y')
plt.xlabel('y (Intensity)')
plt.ylabel('G(H, y)')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# laplace_transform_G(2000, 1950, model_params)
print("Laplace transform:", laplace_transform_G(4.5, 3, model_params))

In [ ]:
select_dominating_intensity_H(3, model_params)

In [ ]:
rho = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 0.99     # Jump size multiplier - paper's value is 2.99 but we use 0.99 for stability 
    # !TODO: Check why Laplace transform blows up with gamma >~ 1.5. 
    # All further calculations of survival probabilities and sampling from f,v are unstable.
}
kappa = model_params["kappa"]
theta = model_params["theta"]
sigma = model_params["sigma"]
y = [0.7, 1.0, 1.5, 2.0, 2.5]
H = [select_dominating_intensity_H(yi, model_params) for yi in y]
for r, y, H in zip(rho, y, H):
    num = hyp1f1(r/kappa, 2*kappa*theta/sigma**2, 2*kappa*y/sigma**2)
    denom = hyp1f1(r/kappa, 2*kappa*theta/sigma**2, 2*kappa*H/sigma**2)
    laplace = num / denom
    print(f"Laplace transform G(H, y) for ρ={r:.2f}: {laplace:.6f}")

In [ ]:
import numpy as np
from scipy.special import hyp1f1
from scipy.optimize import brentq, fsolve
import matplotlib.pyplot as plt

def hypergeom_function(alpha, beta, z):
    """
    Wrapper for the hypergeometric function with error handling
    """
    try:
        return hyp1f1(alpha, beta, z)
    except:
        # Handle overflow or other numerical issues
        return np.inf if alpha > 0 else -np.inf

def find_negative_roots(beta, z, search_range=(-100, 0), num_intervals=100):
    """
    Find negative roots of 1F1(alpha, beta, z) = 0
    """
    # Create a grid to look for sign changes
    alpha_vals = np.linspace(search_range[0], search_range[1], num_intervals)
    func_vals = [hypergeom_function(a, beta, z) for a in alpha_vals]
    
    roots = []
    
    # Look for sign changes to identify bracketing intervals
    for i in range(len(func_vals) - 1):
        if np.sign(func_vals[i]) != np.sign(func_vals[i+1]):
            # Found a sign change, use Brent's method to find the exact root
            try:
                root = brentq(lambda a: hypergeom_function(a, beta, z), 
                            alpha_vals[i], alpha_vals[i+1])
                roots.append(root)
            except ValueError:
                # Brent's method failed, possibly due to numerical issues
                continue
    
    return sorted(roots)

In [ ]:
roota = find_negative_roots(2.5, 3.0)
print("Negative roots found:", roota)

In [ ]:
b = 2*1.61*2.62/0.62**2
z = 2*2.97*2.62/0.62**2

roots = find_negative_roots(b, z)

In [ ]:
print(roots)

In [ ]:
for root in roots:
    print(f"Value at root {root:.6f}: {hypergeom_function(root, b, z)}")

In [ ]:
len(roots)

retrying

In [1]:
import logging
import numpy as np
from scipy.special import hyp1f1, iv
from scipy.optimize import brentq, minimize_scalar
from scipy.integrate import cumulative_trapezoid
from scipy.optimize import toms748
from scipy.integrate import simpson as simps
from scipy.interpolate import interp1d
from typing import Optional, Tuple, Dict, List
from dataclasses import dataclass
import warnings
import time
import math

ModelParams = Dict[str, float]

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('default_simulation.log')
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class SimulationConfig:
    """Configuration for numerical methods and tolerances"""
    # Numerical methods and tolerances
    # Constants for numerical stability and convergence
    eps_zero: float = 1e-12
    eps_small: float = 1e-8
    # Maximum iterations for numerical methods
    max_iterations: int = 1000
    integration_points: int = 500
    root_finding_maxiter: int = 100
    # Number of grid points for numerical integration and interpolation
    grid_points: int = 500
    # Tolerance for numerical methods
    tolerance_scale: float = 1e-6

config = SimulationConfig()

In [2]:
def _hyp1f1(a: float, b: float, z: float) -> float:
    logger.debug(f"Computing hypergeometric function 1F1 with a={a:.6f}, b={b:.6f}, z={z:.6f}")
    return hyp1f1(a, b, z)

In [3]:
import numdifftools as nd

def hyp1f1_derivative_a(a: float, b: float, z: float) -> float:
    """Numerical derivative of hypergeometric function using numdifftools"""
    logger.debug(f"Computing derivative of 1F1 with respect to a={a:.6f}, b={b:.6f}, z={z:.6f}")
    
    try:
        # Create a function that takes only the parameter we want to differentiate
        def f(a_val):
            return _hyp1f1(a_val, b, z)
        
        # Create a derivative function using numdifftools
        df = nd.Derivative(f, method='central', order=8)
        
        # Compute the derivative
        derivative = df(a)
        
        if not np.isfinite(derivative):
            logger.warning(f"Non-finite derivative detected with numdifftools")
            return 0.0
            
        logger.debug(f"Derivative result using numdifftools: {derivative:.6f}")
        return derivative
        
    except Exception as e:
        logger.warning(f"Derivative computation with numdifftools failed: {e}")
        return 0.0

In [4]:
def laplace_transform_G(H: float, y: float, model_params: ModelParams) -> float:
    """Laplace transform computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        a = H / kappa
        b = 2 * kappa * theta / sigma**2
        z1 = 2 * kappa * y / sigma**2
        z2 = 2 * kappa * H / sigma**2
        
        logger.debug(f"Laplace transform: a={a:.6f}, b={b:.6f}, z1={z1:.6f}, z2={z2:.6f}")
        
        numerator = _hyp1f1(a, b, z1)
        denominator = _hyp1f1(a, b, z2)
        
        if abs(denominator) < config.eps_zero:
            logger.warning("Near-zero denominator in Laplace transform")
            return 0.99  # Safe fallback
            
        result = numerator / denominator
        
        # Stability check
        # if not (0 <= result <= 1):
        #     logger.warning(f"Laplace transform out of bounds: {result:.6f}")
        #     result = np.clip(result, config.eps_zero, 1 - config.eps_zero)
            
        return result
        
    except Exception as e:
        logger.error(f"Laplace transform failed: {e}")
        return 0.5  # Conservative fallback

In [5]:
def find_negative_roots(b, z, search_range=(-100, 0), num_intervals=100):
    """
    Find negative roots of 1F1(alpha, b, z) = 0
    """
    logger.debug(f"Finding negative roots of 1F1(alpha, {b:.6f}, {z:.6f}) in range {search_range}")
    
    # Create a grid to look for sign changes
    alpha_vals = np.linspace(search_range[0], search_range[1], num_intervals)
    logger.debug(f"Created grid with {num_intervals} intervals for root finding")
    
    func_vals = [_hyp1f1(a, b, z) for a in alpha_vals]
    
    roots = []
    
    # Look for sign changes to identify bracketing intervals
    for i in range(len(func_vals) - 1):
        if np.sign(func_vals[i]) != np.sign(func_vals[i+1]):
            # Found a sign change, use Brent's method to find the exact root
            try:
                logger.debug(f"Sign change detected between alpha={alpha_vals[i]:.4f} and alpha={alpha_vals[i+1]:.4f}")
                root = brentq(lambda a: _hyp1f1(a, b, z), 
                            alpha_vals[i], alpha_vals[i+1])
                roots.append(root)
                logger.debug(f"Found root at alpha={root:.6f}")
            except ValueError as e:
                logger.warning(f"Brent's method failed for interval [{alpha_vals[i]:.4f}, {alpha_vals[i+1]:.4f}]: {e}")
                continue
    
    logger.info(f"Found {len(roots)} negative roots for 1F1(alpha, {b:.4f}, {z:.4f})")
    return sorted(roots)

In [6]:
def compute_coefficients(y, H, model_params):
    logger.debug(f"Computing series expansion coefficients with y={y:.6f}, H={H:.6f}")
    
    eta_n, beta_n = [], []
    kappa = model_params["kappa"]
    theta = model_params["theta"]
    sigma = model_params["sigma"]

    c_bar = 2 * kappa * theta / sigma**2
    y_bar = 2 * kappa * y / sigma**2
    H_bar = 2 * kappa * H / sigma**2
    
    logger.debug(f"Scaled parameters: c_bar={c_bar:.6f}, y_bar={y_bar:.6f}, H_bar={H_bar:.6f}")

    alpha_n = find_negative_roots(c_bar, H_bar)
    logger.debug(f"Found {len(alpha_n)} roots for coefficient calculation")

    eta_n = [-kappa*alpha for alpha in alpha_n]

    for i, alpha in enumerate(alpha_n):
        numerator = _hyp1f1(alpha, c_bar, y_bar)
        denominator = -alpha*hyp1f1_derivative_a(alpha, c_bar, H_bar)
        
        if denominator == 0:
            logger.warning(f"Zero denominator encountered for root {i}, alpha={alpha:.6f}")
            beta = 0.0
        else:
            beta = numerator / denominator
            
        beta_n.append(beta)
        logger.debug(f"Coefficient {i}: eta={eta_n[i]:.6f}, beta={beta:.6f}")

    logger.info(f"Computed {len(eta_n)} coefficient pairs for hitting probability calculation")
    return eta_n, beta_n

In [7]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 0.99     # Jump size multiplier - paper's value is 2.99 but we use 0.99 for stability 
    # !TODO: Check why Laplace transform blows up with gamma >~ 1.5. 
    # All further calculations of survival probabilities and sampling from f,v are unstable.
}

In [8]:
e, b = compute_coefficients(1.0, 2.0, model_params)
print("Computed coefficients:")
for i, (eta, beta) in enumerate(zip(e, b)):
    print(f"  {i+1}: eta={eta:.6f}, beta={beta:.6f}")

2025-06-13 22:25:10,444 - INFO - Found 25 negative roots for 1F1(alpha, 21.9469, 27.2633)
2025-06-13 22:25:10,459 - INFO - Computed 25 coefficient pairs for hitting probability calculation


Computed coefficients:
  1: eta=261.185568, beta=-0.027990
  2: eta=244.738388, beta=-0.038017
  3: eta=228.767314, beta=-0.013986
  4: eta=213.272589, beta=0.024982
  5: eta=198.254499, beta=0.044022
  6: eta=183.713381, beta=0.023254
  7: eta=169.649635, beta=-0.021752
  8: eta=156.063734, beta=-0.050342
  9: eta=142.956251, beta=-0.032608
  10: eta=130.327875, beta=0.019783
  11: eta=118.179448, beta=0.058641
  12: eta=106.512005, beta=0.041813
  13: eta=95.326835, beta=-0.022055
  14: eta=84.625562, beta=-0.072099
  15: eta=74.410252, beta=-0.049460
  16: eta=64.683586, beta=0.035893
  17: eta=55.449093, beta=0.096285
  18: eta=46.711530, beta=0.046971
  19: eta=38.477463, beta=-0.082735
  20: eta=30.756259, beta=-0.134082
  21: eta=23.561840, beta=0.018829
  22: eta=16.916071, beta=0.226014
  23: eta=10.856069, beta=0.054136
  24: eta=5.452454, beta=-0.577681
  25: eta=0.860924, beta=1.395362


In [9]:
e, b = compute_coefficients(3.7, 5.0, model_params)
print("Computed coefficients:")
for i, (eta, beta) in enumerate(zip(e, b)):
    print(f"  {i+1}: eta={eta:.6f}, beta={beta:.16f}")

2025-06-13 22:25:12,490 - INFO - Found 42 negative roots for 1F1(alpha, 21.9469, 68.1582)
2025-06-13 22:25:12,515 - INFO - Computed 42 coefficient pairs for hitting probability calculation


Computed coefficients:
  1: eta=261.365552, beta=-0.0000170301376079
  2: eta=251.196440, beta=0.0000032666868647
  3: eta=241.218282, beta=0.0000237517602039
  4: eta=231.431175, beta=0.0000399403464156
  5: eta=221.835227, beta=0.0000480398344801
  6: eta=212.430558, beta=0.0000458238091808
  7: eta=203.217300, beta=0.0000332230531700
  8: eta=194.195597, beta=0.0000124840500512
  9: eta=185.365613, beta=-0.0000121728232717
  10: eta=176.727526, beta=-0.0000353602032056
  11: eta=168.281538, beta=-0.0000516560125890
  12: eta=160.027871, beta=-0.0000568374007259
  13: eta=151.966775, beta=-0.0000489491069161
  14: eta=144.098533, beta=-0.0000289492815804
  15: eta=136.423460, beta=-0.0000007434162252
  16: eta=128.941916, beta=0.0000294661417221
  17: eta=121.654305, beta=0.0000544409887924
  18: eta=114.561089, beta=0.0000675552773693
  19: eta=107.662796, beta=0.0000644914209016
  20: eta=100.960030, beta=0.0000445840571283
  21: eta=94.453489, beta=0.0000114369642780
  22: eta=88.

In [10]:
def hitting_probability(y, H, tau, model_params: ModelParams) -> float:
    # logger.debug(f"Computing hitting probability with y={y:.6f}, H={H:.6f}, tau={tau:.6f}")
    
    eta_n, beta_n = compute_coefficients(y, H, model_params)
    # logger.debug(f"Using {len(eta_n)} terms for series expansion")

    valid_terms = [(eta, beta) for eta, beta in zip(eta_n, beta_n) if np.isfinite(beta) and np.isfinite(eta)]
    # if len(valid_terms) < len(eta_n):
        # logger.warning(f"Discarded {len(eta_n) - len(valid_terms)} non-finite terms")
    
    # p = sum(beta * (np.exp(-eta * tau)) for eta, beta in valid_terms)

    p = 0.0

    for eta, beta in valid_terms:
        term = beta * (1-np.exp(-eta * tau))
        # if (abs(term) < config.eps_zero):
        #     logger.debug(f"Term {term:.6f} is near-zero, skipping")
        #     continue
        p += term
        

    return p

In [11]:
hitting_probability(1.0, 2.0, np.random.exponential(1), model_params)

2025-06-13 22:25:14,800 - INFO - Found 25 negative roots for 1F1(alpha, 21.9469, 27.2633)
2025-06-13 22:25:14,817 - INFO - Computed 25 coefficient pairs for hitting probability calculation


np.float64(0.05069478463053856)

In [12]:
hitting_probability(3.7, 4.7, np.random.exponential(1/6.7), model_params)

2025-06-13 22:25:16,909 - INFO - Found 40 negative roots for 1F1(alpha, 21.9469, 64.0687)
2025-06-13 22:25:16,931 - INFO - Computed 40 coefficient pairs for hitting probability calculation


np.float64(-0.00011257498388639188)

In [13]:
def cir_transition_density_g(x: float, y: float, t: float, model_params: ModelParams) -> float:
    """CIR transition density computation"""
    logger.debug(f"Computing CIR transition density for x={x:.6f}, y={y:.6f}, t={t:.6f}")
    
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        if t <= 0 or x < 0 or y < 0:
            logger.warning(f"Invalid inputs for CIR density: t={t}, x={x}, y={y}")
            return 0.0
            
        q = (2 * kappa * theta / sigma**2) - 1
        exp_term = np.exp(-kappa * t)
        a = 2 * kappa / (sigma**2 * (1 - exp_term))
        b = a * exp_term
        
        logger.debug(f"CIR density params: q={q:.6f}, a={a:.6f}, b={b:.6f}")
        
        by = b * y + config.eps_zero
        ax = a * x + config.eps_zero
        
        log_factor = np.log(a) - (ax + b * y)
        log_power = (q / 2) * (np.log(ax) - np.log(by))
        
        bessel_arg = 2 * np.sqrt(a * b * x * y)
        logger.debug(f"Bessel argument: {bessel_arg:.6f}")
        
        if bessel_arg > 100:  
            logger.debug("Using asymptotic approximation for large Bessel argument")
            log_bessel = bessel_arg - 0.5 * np.log(2 * np.pi * bessel_arg)
        else:
            bessel_val = iv(q, bessel_arg)
            if bessel_val <= 0:
                logger.warning(f"Non-positive Bessel value: {bessel_val}")
                return 0.0
            log_bessel = np.log(bessel_val)
        
        log_density = log_factor + log_power + log_bessel
        
        if log_density > 50:  
            logger.warning(f"Excessively large log density: {log_density:.6f}")
            return 0.0
            
        result = np.exp(log_density)
        logger.debug(f"CIR density result: {result:.10f}")
        return result
        
    except Exception as e:
        logger.debug(f"CIR density computation failed: {e}")
        return 0.0

In [14]:
def sample_hitting_time_from_v(y, H, tau, model_params):
    logger.debug(f"Sampling hitting time with y={y:.6f}, H={H:.6f}, tau={tau:.6f}")
    
    p = hitting_probability(y, H, tau, model_params)
    logger.debug(f"Hitting probability until tau: {p:.6f}")
    
    u = np.random.uniform(0, p)
    logger.debug(f"Random uniform sample: u={u:.6f}")
    
    try:
        f = lambda x: hitting_probability(y, H, x, model_params) - u
        logger.debug("Solving for hitting time using toms748 method")
        eta = toms748(f, 0, tau)
        logger.info(f"Sampled hitting time: {eta:.6f}")
        return eta
    except Exception as e:
        logger.error(f"Failed to sample hitting time: {e}")
        return tau / 2  # Return conservative estimate

In [15]:
sample_hitting_time_from_v(3.7, 6.7, np.random.exponential(1/6.7), model_params)


2025-06-13 22:25:21,601 - INFO - Found 49 negative roots for 1F1(alpha, 21.9469, 91.3319)
2025-06-13 22:25:21,627 - INFO - Computed 49 coefficient pairs for hitting probability calculation
2025-06-13 22:25:21,629 - INFO - Found 49 negative roots for 1F1(alpha, 21.9469, 91.3319)
2025-06-13 22:25:21,648 - INFO - Computed 49 coefficient pairs for hitting probability calculation
2025-06-13 22:25:21,651 - INFO - Found 49 negative roots for 1F1(alpha, 21.9469, 91.3319)
2025-06-13 22:25:21,666 - INFO - Computed 49 coefficient pairs for hitting probability calculation
2025-06-13 22:25:21,669 - INFO - Found 49 negative roots for 1F1(alpha, 21.9469, 91.3319)
2025-06-13 22:25:21,684 - INFO - Computed 49 coefficient pairs for hitting probability calculation
2025-06-13 22:25:21,686 - INFO - Found 49 negative roots for 1F1(alpha, 21.9469, 91.3319)
2025-06-13 22:25:21,703 - INFO - Computed 49 coefficient pairs for hitting probability calculation
2025-06-13 22:25:21,706 - INFO - Found 49 negative root

np.float64(0.0017683335064366521)

In [16]:
def sample_conditional_intensity_from_f(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample from conditional transition density"""
    try:
        theta = model_params["theta"]
        nu_scale = max(y, theta, 0.1)
        x_max = max(nu_scale * 5, 10.0)  # Reasonable upper bound
        
        n_points = config.integration_points
        x_grid = np.linspace(config.eps_zero, x_max, n_points, endpoint=False)
        
        
        g_vals = np.array([cir_transition_density_g(x, y, tau, model_params) for x in x_grid])
        
        
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        eta_n = np.array(eta_n)
        beta_n = np.array(beta_n)

        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        def u(s):
            s = np.atleast_1d(s)
            return np.sum(beta_n[:, None] * eta_n[:, None] * np.exp(-eta_n[:, None] * s[None, :]), axis=0)

        
        conv_vals = np.empty_like(x_grid)

        for i, xi in enumerate(x_grid):
            g_shift = np.array([
                cir_transition_density_g(xi, H, tau - sj, model_params)
                for sj in s_grid
            ])
            conv_vals[i] = cumulative_trapezoid(g_shift * u(s_grid), s_grid)[-1]

        p1 = 1 - hitting_probability(y, H, tau, model_params)
            
        f_vals = (g_vals - conv_vals) / p1
        f_vals = np.clip(f_vals, config.eps_zero, None)  
        f_vals /= np.sum(f_vals)  

        if np.sum(f_vals) <= config.eps_zero:
            logger.warning("Sampling failed, using fallback")
            return max(y * np.random.uniform(0.5, 1.5), config.eps_small)
        

        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        
        cdf_vals = np.concatenate(([0.0], cdf_vals))
        total_mass = cdf_vals[-1]
        if total_mass <= 0:
            return y
        cdf_vals /= total_mass

        if cdf_vals[-1] > config.eps_zero:
            cdf_vals /= cdf_vals[-1]
        else:
            
            cdf_vals = np.linspace(0, 1, len(cdf_vals))
        
        
        for i in range(1, len(cdf_vals)):
            cdf_vals[i] = max(cdf_vals[i], cdf_vals[i-1])
        
        
        u = np.random.uniform()
        try:
            inv_cdf = interp1d(cdf_vals, x_grid, bounds_error=False, 
                             fill_value=(x_grid[0], x_grid[-1]))
            sample = float(inv_cdf(u))
        except:
            
            idx = np.searchsorted(cdf_vals, u)
            idx = min(idx, len(x_grid) - 1)
            sample = x_grid[idx]
        
        
        sample = max(sample, config.eps_small)
        
        logger.debug(f"Sampled intensity: {sample:.6f}")
        return sample
        
    except Exception as e:
        logger.error(f"Conditional sampling failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)


In [17]:
def sample_conditional_intensity_f(y, H, tau, model_params):
    """
    Sample ν_τ from the conditional density f(x; y, H, τ)
    using inverse transform method.
    
    Parameters:
        y : float - Starting intensity
        H : float - Dominating threshold
        tau : float - Proposed interarrival time
        model_params : dict - CIR parameters (kappa, theta, sigma)
    
    Returns:
        float : sampled conditional intensity ν_τ
    """
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        x_scale = max(y, theta, 0.1)
        x_max = min(x_scale * 5, 10.0)
        x_grid = np.linspace(config.eps_small, x_max, config.grid_points)

        g_y = np.array([
            cir_transition_density_g(x, y, tau, model_params)
            for x in x_grid
        ])

        eta_n, beta_n = compute_coefficients(y, H, model_params)
        if not eta_n or not beta_n:
            logger.warning("No valid coefficients, fallback to g_y sampling")
            return np.random.choice(x_grid, p=g_y / np.sum(g_y))

        
        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        u_vals = np.sum([
            b * e * np.exp(-e * s_grid) for b, e in zip(beta_n, eta_n)
        ], axis=0)

        conv = np.zeros_like(x_grid)
        for i, x in enumerate(x_grid):
            g_H = np.array([
                cir_transition_density_g(x, H, tau - s, model_params)
                for s in s_grid
            ])
            conv[i] = np.trapezoid(g_H * u_vals, s_grid)

        
        p1 = 1 - hitting_probability(y, H, tau, model_params)
        p1 = max(p1, config.eps_zero)

        
        f_vals = (g_y - conv) / p1
        f_vals = np.clip(f_vals, 0, None)
        f_vals_sum = np.sum(f_vals)

        if f_vals_sum < config.eps_zero:
            logger.warning("Degenerate density in f sampling, fallback")
            return np.random.choice(x_grid)

        
        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        cdf_vals = np.clip(cdf_vals, 0, None)
        cdf_vals /= cdf_vals[-1]

        
        u = np.random.uniform()
        inv_cdf = interp1d(cdf_vals, x_grid, kind='linear',
                           bounds_error=False,
                           fill_value=(x_grid[0], x_grid[-1]))
        x_sample = float(inv_cdf(u))

        return max(x_sample, config.eps_small)

    except Exception as e:
        logger.error(f"Sampling from f failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)


In [18]:
sample_conditional_intensity_f(3.7, 4.7, np.random.exponential(1/4.7), model_params)

2025-06-13 22:25:29,246 - INFO - Found 40 negative roots for 1F1(alpha, 21.9469, 64.0687)
2025-06-13 22:25:29,267 - INFO - Computed 40 coefficient pairs for hitting probability calculation
2025-06-13 22:25:30,653 - INFO - Found 40 negative roots for 1F1(alpha, 21.9469, 64.0687)
2025-06-13 22:25:30,665 - INFO - Computed 40 coefficient pairs for hitting probability calculation


3.6317293358213134

In [19]:
def sample_loss():
    u = np.random.uniform(0, 1)
    if u <= 0.5:
        return 0.24
    else:
        return 0.96

In [20]:
def select_dominating_intensity_H(y: float, model_params: ModelParams, 
                              t_C: float = 1.0, t_eta: float = 0.8, buffer: float = 0.01) -> float:
    """Find optimal threshold H"""
    try:
        theta = model_params["theta"]
        
        H_max = max(y * 3, theta * 2, 1.0)
        H_min = max(y + config.eps_small, theta + buffer, config.eps_small)
        
        def objective(H):
            try:
                G = laplace_transform_G(H, y, model_params)
                
                numerator = H * (t_C + G * t_eta)
                denominator = 1 - G
                
                if denominator < config.eps_zero:
                    return 1e10
                    
                return numerator / denominator 
                
            except:
                return 1e10
        
        result = minimize_scalar(objective, bounds=(H_min, H_max), method='bounded')
        
        if not result.success:
            logger.warning("H optimization failed, using heuristic")
            H_star = max(y * 1.5, theta * 1.2)
        else:
            H_star = result.x
        
        # Apply safety constraints
        H_final = max(H_star, y + config.eps_small, theta + buffer)
        
        logger.debug(f"Optimal H: {H_final:.6f} (y={y:.6f})")
        return H_final
        
    except Exception as e:
        logger.error(f"H optimization failed: {e}")
        return max(y * 1.5, theta * 1.2, 1.0)


In [23]:
def simulate_next_default_step(
    t_prev: float,
    lambda_prev: float,
    T_max: float,
    model_params: ModelParams
) -> Optional[Tuple[float, float]]:
    """
    Simulates a single default step in the top-down model.

    Parameters:
        t_prev: float
            Previous default time T_{n-1}
        lambda_prev: float
            Intensity after last jump, λ_{T_{n-1}} 
        T_max: float
            Final time horizon for simulation
        model_params: dict
            Contains CIR and model parameters: kappa, theta, sigma, gamma

    Returns:
        Optional[Tuple[float, float]]:
            If a default occurs before T_max:
                (T_n, ν_{T_n})
            Else:
                None (no more defaults)
    """
    try:
        # STEP 1: Initialize y := λ_{T_{n-1}}, t := T_{n-1}
        y = max(lambda_prev, config.eps_small) # Ensure y is positive
        t = t_prev 
        
        max_inner_iterations = 50  
        
        for iteration in range(max_inner_iterations):
            # STEP 2: Select H>y
            # H = y + model_params["theta"]*np.random.uniform(1.1, 1.3) + 0.1
            H = select_dominating_intensity_H(y, model_params)
            logger.info(f"Iteration {iteration + 1}: Selected H={H:.6f} for y={y:.6f}")
            
            # STEP 3: Sample τ ∼ Exp(H), propose next interarrival time
            tau = np.random.exponential(1/H)
            logger.debug(f"Sampled interarrival time τ={tau:.6f} from Exp(H) with H={H:.6f}")
            
            # STEP 4: Compute survival probability p1 = P(ν stays below H up to τ)
            p1 = 1 - hitting_probability(y, H, tau, model_params)
            logger.info(f"Computed survival probability p1={p1:.6f} for y={y:.6f}, H={H:.6f}, τ={tau:.6f}")
            
            # STEP 5: First rejection test with u1 ∼ Unif(0,1)
            u1 = np.random.uniform()
            
            if u1 > p1:
                # Process hits H → sample σ_H from hitting time density
                logger.info(f"Process hits boundary H={H:.6f} at t={t:.6f}, sampling hitting time")
                sigma_H = sample_hitting_time_from_v(y, H, tau, model_params)
                t += sigma_H
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                    
                y = H  # Update intensity and repeat
                continue
            else:
                # Process survives to τ
                t += tau
                logger.info(f"Process survives to t={t:.6f}, sampling conditional intensity")
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                
                # STEP 6: Sample ν_{T_n} from conditional density f(.; ν_t, H, τ)
                nu_Tn = sample_conditional_intensity_from_f(y, H, tau, model_params)
                logger.debug(f"Sampled conditional intensity ν_{t}={nu_Tn:.6f} at t={t:.6f}")
                
                # STEP 7: Second rejection test using u2 ∼ Unif(0,1)
                u2 = np.random.uniform()
                
                if u2 <= nu_Tn / H:
                    # Accept default at T_n = t
                    # ell_n = sample_loss()  # Draw loss ℓ_n ∼ loss_distribution()
                    
                    # logger.debug(f"Default at t={t:.6f}, ν={nu_Tn:.6f}, loss={ell_n:.6f}")
                    logger.info(f"Default accepted at t={t:.6f}, ν={nu_Tn:.6f}.")
                    return (t, nu_Tn)
                else:
                    # Reject - update ν_t and go back to step 2
                    y = nu_Tn
                    continue
        
        logger.warning(f"Maximum inner iterations reached at t={t:.6f}")
        return None
        
    except Exception as e:
        logger.error(f"Error in simulate_next_default_step: {e}")
        return None

def simulate_defaults(
    lambda_0: float,
    T_max: float,
    model_params: ModelParams,
    max_defaults: int = 1000
) -> List[Tuple[float, float]]:
    """
    Main simulation function that generates a sequence of default times and losses.
    
    Parameters:
        lambda_0: float
            Initial intensity λ_0
        T_max: float
            Time horizon
        model_params: ModelParams
            Dictionary containing kappa, theta, sigma, gamma
        max_defaults: int
            Maximum number of defaults to simulate
            
    Returns:
        List[Tuple[float, float]]: List of (default_time, loss) pairs
    """
    logger.info(f"Starting default simulation: λ_0={lambda_0:.6f}, T_max={T_max:.2f}")
    
    # Validate parameters
    if not validate_parameters(model_params):
        logger.warning("Proceeding despite Feller condition violation")
    
    defaults = []
    t = 0.0
    lam = max(lambda_0, config.eps_small)
    # gamma = model_params.get("gamma", 2.99)  # Default jump size
    gamma = model_params["gamma"]
    start_time = time.time()
    
    try:
        while t < T_max and len(defaults) < max_defaults:
            result = simulate_next_default_step(t, lam, T_max, model_params)
            
            if result is None:
                break
                
            Tn, nu_Tn = result
            ell_n = sample_loss()
            lam = nu_Tn + gamma * ell_n  # Update intensity at default with jump
            t = Tn
            defaults.append((t, lam, ell_n))
            
            if len(defaults) % 10 == 0:
                logger.info(f"Generated {len(defaults)} defaults, t={t:.4f}, λ={lam:.6f}")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Simulation completed: {len(defaults)} defaults in {elapsed_time:.2f}s")
        
        return defaults
        
    except KeyboardInterrupt:
        logger.info("Simulation interrupted by user")
        return defaults
    except Exception as e:
        logger.error(f"Critical simulation error: {e}")
        return defaults

In [24]:
# paper parameters for testing
model_params = {
    "kappa": 2.62,    # Mean reversion speed
    "theta": 1.61,    # Long-term mean
    "sigma": 0.62,    # Volatility
    "gamma": 2.99     # Jump size multiplier
}

defaults = simulate_defaults(
    lambda_0=0.7,
    T_max=5,
    model_params=model_params,
    max_defaults=100
)

print(f"Generated {len(defaults)} defaults:")
for i, (t, lam, loss) in enumerate(defaults): 
    print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

L = sum(loss for _, _, loss in defaults)
print(f"Total loss: {L:.6f}")

2025-06-13 22:26:07,107 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-06-13 22:26:07,108 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-06-13 22:26:07,109 - INFO - Feller condition satisfied
2025-06-13 22:26:07,110 - INFO - Iteration 1: Selected H=2.188060 for y=0.700000
2025-06-13 22:26:07,112 - INFO - Found 26 negative roots for 1F1(alpha, 21.9469, 29.8268)
2025-06-13 22:26:07,129 - INFO - Computed 26 coefficient pairs for hitting probability calculation
2025-06-13 22:26:07,129 - INFO - Computed survival probability p1=0.805794 for y=0.700000, H=2.188060, τ=0.737222
2025-06-13 22:26:07,130 - INFO - Process survives to t=0.737222, sampling conditional intensity
2025-06-13 22:26:07,136 - INFO - Found 26 negative roots for 1F1(alpha, 21.9469, 29.8268)
2025-06-13 22:26:07,147 - INFO - Computed 26 coefficient pairs for hitting probability calculation
2025-06-13 22:26:08,674 - INFO - Found 26 negative roots for 1F1(alpha, 21.9469, 29.8268)
2025-06-13 22:

Generated 20 defaults:
  1: t=0.737222, itensity=1.777600, loss=0.240000
  2: t=0.784788, itensity=2.517600, loss=0.240000
  3: t=0.862838, itensity=3.411432, loss=0.240000
  4: t=0.939809, itensity=3.856117, loss=0.240000
  5: t=2.767739, itensity=5.360744, loss=0.960000
  6: t=2.853120, itensity=5.435055, loss=0.240000
  7: t=2.953733, itensity=7.816300, loss=0.960000
  8: t=3.054950, itensity=8.366041, loss=0.960000
  9: t=3.120946, itensity=11.403761, loss=0.960000
  10: t=4.110449, itensity=5.009175, loss=0.960000
  11: t=4.329981, itensity=4.524573, loss=0.240000
  12: t=4.457522, itensity=6.263830, loss=0.960000
  13: t=4.477613, itensity=8.946315, loss=0.960000
  14: t=4.505887, itensity=9.663915, loss=0.240000
  15: t=4.517037, itensity=12.147758, loss=0.960000
  16: t=4.531442, itensity=15.382591, loss=0.960000
  17: t=4.713007, itensity=10.293263, loss=0.240000
  18: t=4.748951, itensity=10.393267, loss=0.240000
  19: t=4.762153, itensity=13.263667, loss=0.960000
  20: t=4.9

In [ ]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 2.99,     # Jump size multiplier
    "true": 12.17
}

L_5 = []

n_runs = 15
for _ in range(n_runs):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=100
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over {n_runs} runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")

